# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number.

Each row of the dataset is a list of integer number pairs, followed by a string which is the label of the document (Joy or sadness).
The first number of the pair is an identifier of an element (word, emoji or emoticon) and the second number of the pair is the count (frequency) of that element in that document.

The dataset has:
tot_n_docs=n_rows=11981
n_features (document elements)=11288



The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library

In [22]:

from numpy import ndarray, zeros
import numpy as np
import scipy
from scipy.sparse import csr_matrix

class_labels = ["Joy","Sadness"]
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows=11981 # number rows of the matrix
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

#path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training="datasets/"
file_name="joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int)
col=np.empty(n_elements, dtype=int)
data=np.empty(n_elements, dtype=int)

row_n=0 # number of current row to be read and managed
cur_el=0 # position in the arrays row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')
        l=len(el_list)
        last_el=el_list[l-1] # I grab the last element in the list which is the class label
        class_name=last_el.strip() # eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)


final n_row=[0 0 0 ... 1 1 1]
resulting matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
['Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy', 'Joy'

Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.

In [23]:
from sklearn import model_selection
from sklearn import naive_bayes as nb

## Splitting the dataset

In [24]:
train_size = 0.8
arr_indices = np.arange(len(twitter_target))
np.random.shuffle(arr_indices)

n_train_examples = int(np.round(len(twitter_target)*train_size))
train_indices = arr_indices[0:n_train_examples]
test_indices = arr_indices[n_train_examples:]

X_train = twitter_data[train_indices] 
y_train = [twitter_target[i] for i in train_indices] 
X_test = twitter_data[test_indices] 
y_test = [twitter_target[i] for i in test_indices]

In [25]:
if 2 in X_test[1000]:
    print("Yay")

## Fitting and prediction

In [35]:
multivariate_classifier = nb.BernoulliNB(binarize = 0.1)
multivariate_classifier.fit(X_train, y_train)

multivariate_classifier.score(X_test, y_test)



0.9545075125208681

In [37]:
multimodal_classifier = nb.MultinomialNB()
multimodal_classifier.fit(X_train, y_train)

multimodal_classifier.score(X_test, y_test)

0.9528380634390651

## Cross-validation with multiple metrics
Multiple metrics are evaluated for both classifiers using cross-validation


In [28]:
# prints the results of the evaluation
def print_test_results(title, scoring, scores):
    print(title)
    for s in scoring:
        test_name = "test_" + s
        print("  " + s.capitalize() + ":\t" + str(scores[test_name]))
    print()

In [29]:
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'f1_macro', 'recall', 'roc_auc']
k_folds = 10

mv_scores = cross_validate(multivariate_classifier, twitter_data, twitter_target, cv=k_folds, scoring=scoring)
print_test_results("Multivariate scores", scoring, mv_scores)

mm_scores = cross_validate(multimodal_classifier, twitter_data, twitter_target, cv=k_folds, scoring=scoring)
print_test_results("Multimodal scores", scoring, mm_scores)

Multivariate scores
  Accuracy:	[0.95663053 0.96577629 0.94991653 0.94991653 0.94824708 0.9524207
 0.95075125 0.95492487 0.9624374  0.93656093]
  F1_macro:	[0.95661456 0.96576768 0.94988915 0.94987615 0.94818932 0.95237527
 0.95072622 0.95489659 0.96240887 0.93649704]
  Recall:	[0.975      0.98163606 0.97328881 0.97829716 0.98163606 0.98330551
 0.97328881 0.97996661 0.98833333 0.96666667]
  Roc_auc:	[0.99587646 0.99757804 0.9949805  0.99571071 0.99544873 0.99608697
 0.99352845 0.99578875 0.99691193 0.99191193]

Multimodal scores
  Accuracy:	[0.95412844 0.96494157 0.94991653 0.94824708 0.94407346 0.94741235
 0.9490818  0.94991653 0.96160267 0.93739566]
  F1_macro:	[0.95410341 0.9649318  0.94988077 0.94819496 0.94401412 0.94736743
 0.94904769 0.94986063 0.96157858 0.9373149 ]
  Recall:	[0.97666667 0.98163606 0.97662771 0.97996661 0.97662771 0.97662771
 0.97495826 0.98330551 0.985      0.97166667]
  Roc_auc:	[0.99574569 0.99720458 0.99378764 0.99560481 0.99372633 0.99415275
 0.99370124 0.

### Mean accuracy and standard deviation
 The mean accuracy and the standard deviation are calculated upon the scores obtained by cross-validation 

In [30]:
import statistics
from statistics import stdev

mm_accuracy_scores = mm_scores['test_accuracy']
mv_accuracy_scores = mv_scores['test_accuracy']

mm_acc_mean = mm_accuracy_scores.mean()
mv_acc_mean = mv_accuracy_scores.mean()

mm_stdev = stdev(mm_accuracy_scores)
mv_stdev = stdev(mv_accuracy_scores)

print("MM mean accuracy: " + str(mm_acc_mean) + " with stdev " + str(mm_stdev))

print("MV mean accuracy: " + str(mv_acc_mean) + " with stdev " + str(mv_stdev))

MM mean accuracy: 0.9506716086443767 with stdev 0.007993225623264152
MV mean accuracy: 0.9527582111414491 with stdev 0.00806804770487353


The same computation is made upon the difference of the scores obtained by the two classifiers

In [31]:
acc_diff = mv_accuracy_scores - mm_accuracy_scores
mean_diff = mv_acc_mean - mm_acc_mean
stdev_diff = mv_stdev - mm_stdev

print("\t\tMV-MM:")
print()
for d in acc_diff:
    print("\t\t" + str(d))
print()
print("Avg diff:\t" + str(mean_diff))
print("Stdev diff:\t" + str(stdev_diff))

		MV-MM:

		0.002502085070892335
		0.0008347245409014992
		0.0
		0.0016694490818029983
		0.004173622704507496
		0.005008347245408995
		0.0016694490818029983
		0.005008347245408995
		0.0008347245409014992
		-0.0008347245409014992

Avg diff:	0.002086602497072376
Stdev diff:	7.482208160937767e-05


## Paired t-test

In [32]:
import scipy
from scipy.stats import ttest_rel

In [33]:
# significancy
a = 0.05

t_stat, p_value = ttest_rel(mm_accuracy_scores, mv_accuracy_scores)

print("p-value: " + str(p_value))

tbd = "can't" if p_value < a else "can"
print('We ' + str(tbd) + ' refuse the null hypotesis that the classifiers perform in the same way.')

p-value: 0.010617994606921194
We can't refuse the null hypotesis that the classifiers perform in the same way.


In [34]:
t_stat

-3.2123990690558073